In [ ]:
import sys
import deepHSI

from pathlib import Path

import numpy as np
from lightning.pytorch import Trainer, seed_everything

# Custom module imports
from deepHSI.datamodule.components import HyperspectralDataset
from deepHSI.datamodule.components.utils import *

from deepHSI.datamodule.medical_datasets import BloodDetectionHSIDataModule

from deepHSI.datamodule.remote_sensing_datasets import *
# from deepHSI.datamodule.remote_sensing_datasets.paviaC import PaviaCDataModule

from deepHSI.models.architectures import HSIFCModel, \
    SpectralSpatialCNN, HyperspectralCNNDetector
from deepHSI.models.task_algos import HSIClassificationModule

seed_everything(42, workers=True)

# Importing from `lightning` instead of `pytorch_lightning`
import lightning as L

# PyTorch and metrics imports
import torch
from torchmetrics import F1Score, Precision, Recall

# from lightning import Trainer

torch.set_float32_matmul_precision("medium")

In [ ]:
from pathlib import Path

# Specify the directory to save checkpoints
ckpt_dir = Path("/home/sayem/Desktop/deepHSI/notebooks/ckpt")

# Function to clear directory

def clear_directory(path: Path):
    if path.is_dir():
        for item in path.iterdir():
            if item.is_dir():
                clear_directory(item)
                item.rmdir()
            else:
                item.unlink()
    else:
        path.mkdir(parents=True, exist_ok=True)


# Clear and/or create the log and checkpoint directories
clear_directory(ckpt_dir)

In [ ]:
# Define the parameters for the data module
data_dir = "/home/sayem/Desktop/deepHSI/data"  # Specify the directory where you want the data to be downloaded

# Include 'batch_size', 'num_workers', and 'num_classes' within the hyperparams dictionary
hyperparams = {
    "batch_size": 64,
    "num_workers": 24,
    "patch_size": 10,
    "center_pixel": True,
    "supervision": "full",
    "num_classes": 10,  # Define the number of classes in your dataset
}

# Assuming YourModel is defined elsewhere and num_classes is known
input_channels = 102

# Define custom metrics for the classification task using the updated hyperparams
custom_metrics = {
    "precision": Precision(
        num_classes=hyperparams["num_classes"], average="macro", task="multiclass"
    ),
    "recall": Recall(num_classes=hyperparams["num_classes"], average="macro", task="multiclass"),
    "f1": F1Score(num_classes=hyperparams["num_classes"], average="macro", task="multiclass"),
}

# model = HSIFCResNetModel(
#     input_channels=input_channels,
#     patch_size=hyperparams["patch_size"],  # Use patch_size from hyperparams
#     n_classes=hyperparams["num_classes"],  # Use num_classes from hyperparams
#     dropout=True,
# )

model = HyperspectralCNNDetector(
    input_channels=input_channels,
    patch_size=hyperparams["patch_size"],  # Use patch_size from hyperparams
    n_classes=hyperparams["num_classes"],  # Use num_classes from hyperparams
    dropout=True,
)

In [ ]:
from deepHSI.datamodule.transforms.hsi_transforms import *

# Instantiate individual transformations
normalize_transform = HSINormalize()
flip_transform = HSIFlip()
rotate_transform = HSIRotate()
noise_transform = HSISpectralNoise(mean=0.0, std=0.01)
shift_transform = HSISpectralShift(shift=2)
drop_transform = HSIRandomSpectralDrop(drop_prob=0.1)

# Compose transformations together
composed_transform = Compose([
    normalize_transform,
    flip_transform,
    # rotate_transform,
    noise_transform,
    shift_transform,
    drop_transform
])

In [ ]:
import torch
from torch.optim import Adam, lr_scheduler

# Define the parameters for the optimizer
optimizer_params = {
    'lr': 0.001,
    'betas': (0.9, 0.999),
    'eps': 1e-08,
    'weight_decay': 0,
}

# Define the parameters for the scheduler
scheduler_params = {
    'step_size': 10,
    'gamma': 0.1,
}

In [ ]:
import torch
from torch.optim import Adam, lr_scheduler

# Initialize the HSIClassificationModule with the model and other hyperparameters
hsi_classifier = HSIClassificationModule(
    net=model,  # Your model instance
    optimizer_constructor=Adam,
    optimizer_params=optimizer_params,
    scheduler_constructor=lr_scheduler.StepLR,
    scheduler_params=scheduler_params,
    loss_fn=torch.nn.functional.cross_entropy,  # Using cross-entropy loss function
    num_classes=hyperparams["num_classes"],  # Number of classes from your hyperparameters
    custom_metrics=custom_metrics,  # Custom metrics dictionary if any
)

# # Initialize the PyTorch Lightning Trainer
# trainer = Trainer(max_epochs=10, precision='16-mixed', accelerator='gpu', devices=1)
max_epochs = 200

# Initialize the PaviaCDataModule with the updated arguments
pavia_c_datamodule = PaviaCDataModule(
    data_dir=data_dir, hyperparams=hyperparams, \
    transform=composed_transform,  # Pass hyperparams which now includes num_classes
)

In [ ]:
import os
import wandb
from pytorch_lightning.loggers import WandbLogger

# Extract the types (names) of the optimizer and scheduler from the module
optimizer_type = hsi_classifier.optimizer_constructor.__name__  # E.g., 'Adam'
scheduler_type = hsi_classifier.scheduler_constructor.__name__  # E.g., 'StepLR'

# Dynamically get the model name from the module
model_name = hsi_classifier.net.__class__.__name__  # Assuming the BaseModule has a 'model' attribute

# Set the WandB notebook name for better organization
os.environ["WANDB_NOTEBOOK_NAME"] = "1.0-hsi-initial-data-exploration.ipynb"
wandb.login()

# Dynamically construct the run name to include key experiment details
dynamic_run_name = f"{optimizer_type}-with-{scheduler_type}-{model_name}"

# Extend the dynamic tags to include other important aspects of the experiment
dynamic_tags = [optimizer_type, scheduler_type, model_name]
# Add more tags as needed, e.g., dataset name, special model configurations

# Include additional notes for the experiment to describe unique aspects or goals
experiment_notes = f"Testing {optimizer_type} optimizer with {scheduler_type} scheduler on {model_name}."

# Initialize WandbLogger with dynamic configurations
wandb_logger = WandbLogger(
    name=dynamic_run_name,
    project="PaviaC",
    save_dir="/home/sayem/Desktop/deepHSI/notebooks/wandb",
    offline=False,
    tags=dynamic_tags,
    notes=experiment_notes,
)

# Update `hyperparams` to include optimizer, scheduler, and model details
hyperparams.update({
    "optimizer": {
        "type": optimizer_type,
        "params": hsi_classifier.optimizer_params  # Assuming this is how you access optimizer parameters
    },
    "scheduler": {
        "type": scheduler_type,
        "params": hsi_classifier.scheduler_params  # Assuming this is how you access scheduler parameters
    },
    "model": model_name,
})

# Update the WandB config with the comprehensive set of experiment details
wandb_logger.experiment.config.update(hyperparams)

In [ ]:
# Callbacks: Pytorch Callbacks: See L doc
# Define the EarlyStopping callback
early_stop_callback = L.pytorch.callbacks.EarlyStopping(
    monitor="val/f1",  # Specify the metric to monitor
    patience=20,  # Number of epochs with no improvement after which training will be stopped
    verbose=True,  # Whether to print logs to stdout
    mode="max",  # In 'min' mode, training will stop when the quantity monitored has stopped decreasing
    check_on_train_epoch_end=False,
)

# from lightning.pytorch.callbacks import ModelCheckpoint

# Define the ModelCheckpoint callback
model_checkpoint = L.pytorch.callbacks.ModelCheckpoint(
    monitor="val/f1",  # Metric to monitor
    dirpath=str(ckpt_dir),  # Convert Path object to string, Directory to save checkpoints
    filename="best-checkpoint-{epoch:02d}-{val/f1:.2f}",  # Checkpoint file name
    save_top_k=1,  # Save only the best checkpoint
    mode="max",  # 'max' because we want to maximize 'val/f1'
    verbose=True,  # Print a message when a new best is found
    auto_insert_metric_name=False,  # Prevents metric names being inserted into filename automatically
)

rich_pbar_callback = L.pytorch.callbacks.RichProgressBar(
    refresh_rate=1,
    leave=True,
)

lr_monitor_callback = L.pytorch.callbacks.LearningRateMonitor(logging_interval='epoch') 

In [ ]:
# Initialize the PyTorch Lightning Trainer with fast_dev_run enabled
trainer = L.Trainer(
    fast_dev_run=False,  # Enable fast_dev_run
    precision="16-mixed",  # Use 16-bit precision
    accelerator="auto",  # Specify the accelerator as GPU
    max_epochs=max_epochs,
    log_every_n_steps=3,
    callbacks=[
        # lr_finder_callback,
        early_stop_callback,
        model_checkpoint,
        # confusion_matrix_callback,
        # batch_finder_callback,
        lr_monitor_callback,
        # model_weight_logger_callback,
    ],  # rich_pbar_callback],
    logger=wandb_logger,
    deterministic=False,
)

In [ ]:
trainer.fit(hsi_classifier, datamodule=pavia_c_datamodule)

In [ ]:
# Fit the model using the train dataset from the data module
dictionary = trainer.test(hsi_classifier, pavia_c_datamodule, verbose=True)
# trainer.fit(hsi_module, datamodule=pavia_c_datamodule)tree
# Use train_dataloader() instead of train_dataset

In [ ]:
# After training or experiments are done, finish the WandB run
wandb.finish()